

# Exploring the Neighborhoods in Singapore: Data Science in Real Life 

As a part of the final IBM Capstone Project, we get a to know of what data scientists go through in real life. Objectives of the final assignments were to define a business problem, look for data in the web and, use Foursquare location data to compare different districts within wards (municipalities) of Singapore (choice of city depends on the students) to figure out which neighborhood is suitable for starting a restaurant business(‘idea’ also depends on individual students).

Before we get the data and start exploring it, let's download all the dependencies that we will need.

### Table of Contents

#### 1. Discussion and Background of the Business Problem: 

#### 2. Data Preparation:

#### 3. Explore regions in Singapore

#### 4. Analyze Each Neighborhood

#### 5. Cluster Neighborhoods

#### 6. Examine Clusters

#### 7. Results and Discussion:

#### 8. Conclusion


In [ ]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # c

!pip install beautifulsoup4

!python3 -m pip install lxml

Solving environment: | 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Discussion and Background of the Business Problem: 

### Problem Statement: Prospects of a Lunch Restaurant, Singapore.

Singapore’s economic freedom score is 89.4, making its economy the 2nd freest in the 2019 Index. Its overall score has increased by 0.6 point, with increases in scores for trade freedom and government integrity outpacing modest declines in labor freedom and property rights. Singapore is ranked 2nd among 43 countries in the Asia–Pacific region, and its overall score is well above the regional and world averages.

The aim of this project is to explore the areas of singapore and find the best place to open a breakfast cum lunch restuarant

### Target Audience

What type of clients or a group of people would be interested in this project?
    
    
1.Business personnel who wants to invest or open a restaurant. This analysis will be a comprehensive guide to start or expand restaurants targeting the large pool of office workers in singapore during lunch hours.
    
2.Freelancer who loves to have their own restaurant as a side business. This analysis will give an idea, how beneficial it is to open a restaurant and what are the pros and cons of this business.
    
3.New graduates, to find reasonable lunch/breakfast place close to office.
    
4.Budding Data Scientists, who want to implement some of the most used Exploratory Data Analysis techniques to obtain necessary data, analyze it, and, finally be able to tell a story out of it.

## 2. Data Preparation:

### 2.1. Get The Names of Wards, Major Districts and Population from Wikipedia 

In [10]:
from bs4 import BeautifulSoup

response_obj = requests.get('https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore').text
print (type (response_obj))

<class 'str'>


In [11]:
soup = BeautifulSoup(response_obj,'html.parser')
print (soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Planning Areas of Singapore - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Planning_Areas_of_Singapore","wgTitle":"Planning Areas of Singapore","wgCurRevisionId":904772200,"wgRevisionId":904772200,"wgArticleId":2224605,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using deprecated image syntax","Urban planning in Singapore","Subdivisions of Singapore"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","

### Processing the Information From Wiki To Make Necessary Lists


In [12]:
#pinpointing the location of the table and its contents
Wards_Singpore_Table = soup.find('table', class_ = 'wikitable sortable')
Wards_Singpore_Table

<table class="wikitable sortable">
<tbody><tr>
<th>Name <small>(<a href="/wiki/English_language" title="English language">English</a>)</small>
</th>
<th><a href="/wiki/Malay_language" title="Malay language">Malay</a>
</th>
<th><a href="/wiki/Chinese_language" title="Chinese language">Chinese</a>
</th>
<th><a href="/wiki/Pinyin" title="Pinyin">Pinyin</a>
</th>
<th><a href="/wiki/Tamil_language" title="Tamil language">Tamil</a>
</th>
<th>Region
</th>
<th>Area (km2)
</th>
<th>Population<sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[6]</a></sup>
</th>
<th>Density (/km2)
</th></tr>
<tr>
<td><a href="/wiki/Ang_Mo_Kio" title="Ang Mo Kio">Ang Mo Kio</a>
</td>
<td>
</td>
<td>宏茂桥
</td>
<td>Hóng mào qiáo
</td>
<td>ஆங் மோ கியோ
</td>
<td><a href="/wiki/North-East_Region,_Singapore" title="North-East Region, Singapore">North-East</a>
</td>
<td>13.94
</td>
<td>165,710
</td>
<td>12,000
</td></tr>
<tr>
<td><a href="/wiki/Bedok" title="Bedok">Bedok</a>
</td>
<td>*
</td>
<td>勿洛
</td>
<td>

In [13]:
Name=[]
Region = []
Area = []
Population = []
Density = []

for row in Wards_Singpore_Table.findAll("tr"):
    #print (row)    
    Ward = row.findAll('td')
    #print (len(Ward))
   
    if len(Ward)==9: #Only extract table body not heading
        Name.append(Ward[0].find(text=True).rstrip())
        Region.append(Ward[5].find(text=True).rstrip())
        Area.append(Ward[6].find(text=True).rstrip())
        Population.append(Ward[7].find(text=True).rstrip())
        Density.append(Ward[8].find(text=True).rstrip())
            


In [14]:

Singapore_data=pd.DataFrame(Name,columns=['Name'])
Singapore_data['Region']=Region
Singapore_data['Area_SqKm']=Area
Singapore_data['Population']=Population
Singapore_data['Density_Per_SqKm']=Density
Singapore_data


,Name,Region,Area_SqKm,Population,Density_Per_SqKm
0,Ang Mo Kio,North-East,13.94,"165,710","12,000"
1,Bedok,East,21.69,"281,300","13,000"
2,Bishan,Central,7.62,"88,490","12,000"
3,Boon Lay,West,8.23,30,3.6
4,Bukit Batok,West,11.13,"144,410","13,000"
5,Bukit Merah,Central,14.34,"151,870","11,000"
6,Bukit Panjang,West,8.99,"140,820","16,000"
7,Bukit Timah,Central,17.53,"77,280","4,400"
8,Central Water Catchment,North,37.15,*,*
9,Changi,East,40.61,"2,080",62.3


In [15]:
Singapore_data_Final=pd.DataFrame(Name,columns=['Name'])
Singapore_data_Final['Region']=Region
Singapore_data_Final['Area_SqKm']=Area
Singapore_data_Final.index = np.arange(1, len(Singapore_data_Final) + 1) # reset the index so that it starts from 1. 

Singapore_data_Final

,Name,Region,Area_SqKm
1,Ang Mo Kio,North-East,13.94
2,Bedok,East,21.69
3,Bishan,Central,7.62
4,Boon Lay,West,8.23
5,Bukit Batok,West,11.13
6,Bukit Merah,Central,14.34
7,Bukit Panjang,West,8.99
8,Bukit Timah,Central,17.53
9,Central Water Catchment,North,37.15
10,Changi,East,40.61


Get the Coordinates of the Major Districts


In [16]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
Singapore_data_Final['Area_Name_Coord']= Singapore_data_Final['Name'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

Singapore_data_Final

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


GeocoderUnavailable: Service not available

In [ ]:
Singapore_data_Final[['Latitude', 'Longitude']] = Singapore_data_Final['Area_Name_Coord'].apply(pd.Series)
Singapore_data_Final

In [ ]:
Singapore_data_Final.drop(['Area_Name_Coord'], axis=1, inplace=True)
Singapore_data_Final

## We have the Dataframe with Coordinates 

But here we see problem with coordinates for some places like Bukit Merah,Central Water Catchment,Changi,Downtown Core,Mandai,Museum,Newton,North-Eastern Islands,Orchard,Outram,Pioneer,Queenstown,River Valley,Simpang,Tengah,Western Islands,Woodlands. So we need to replace them manually

Google search gives the values

Bukit Merah---1.2819054° N,103.8217113° E

Central Water Catchment---1.3552054° N,103.7950113° E

Changi---1.3450054° N,103.9810113° E

Downtown Core---1.2867054° N,103.8513113° E

Mandai---1.4260054° N,103.8219113° E

Museum---1.2960317° N,103.8424705° E

Newton---1.3076054° N,103.8382113° E

North-Eastern Islands---1.4064054° N,104.0301113° E

Orchard---1.3048054° N,103.8296113° E

Outram---1.2849054° N,103.8417113° E

Pioneer---1.33587865648° N,103.691660567° E

Queenstown---1.2942054° N,103.7839113° E

River Valley---1.2959054° N,103.8339113° E

Simpang---1.4443054° N,103.8406113° E

Tengah---1.3555054° N,103.7286113° E

Western Islands---1.2479054° N,103.6746113° E

Woodlands---1.4382054° N,103.7868113° E


In [ ]:
index_to_change=[6,9,10,14,21,25,26,27,29,30,33,35,36,42,49,52,54]
latitude_to_change=[1.2819054,1.3552054,1.3450054,1.2867054,1.4260054,1.2960317,1.3076054,1.4064054,1.3048054,1.2849054,1.3358787,1.2942054,1.2959054,1.4443054,1.3555054,1.2479054,1.4382054]
longitude_to_change=[103.8217113,103.7950113,103.9810113,103.8513113,103.8219113,103.8424705,103.8382113,104.0301113,103.8296113,103.8417113,103.691661,103.7839113,103.8339113,103.8406113,103.7286113,103.6746113,103.7868113]

for i in range(len(index_to_change)):
    print('Before',Singapore_data_Final.loc[index_to_change[i],'Latitude'],Singapore_data_Final.loc[index_to_change[i],'Longitude'])
    Singapore_data_Final.loc[index_to_change[i],'Latitude']= latitude_to_change[i]
    Singapore_data_Final.loc[index_to_change[i],'Longitude']= longitude_to_change[i]
    print('After',Singapore_data_Final.loc[index_to_change[i],'Latitude'],Singapore_data_Final.loc[index_to_change[i],'Longitude'])
    
    

### Final Data-Frame with Coordinates of the Major Areas

In [ ]:
Singapore_data_Final

In [ ]:
#Singapore_data_Final = Singapore_data_Final.groupby('Region').agg({'BOUROUGH':'first', 
#                             'NEIGHBOURHOOD': ', '.join}).reset_index()
#Singapore_data_Final = Singapore_data_Final.replace('\n','', regex=True)
#Singapore_data_Final

#### Use geopy library to get the latitude and longitude values of Singapore.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [ ]:
address = 'Singapore'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore City are {}, {}.'.format(latitude, longitude))

#### Create a map of Singapore with neighborhoods superimposed on top.

In [ ]:
# create map of Toronto using latitude and longitude values
map_singapore= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng , Name , Region in zip(Singapore_data_Final['Latitude'], Singapore_data_Final['Longitude'], Singapore_data_Final['Name'], Singapore_data_Final['Region']):
    label = '{}, {}'.format(Region, Name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_singapore)  
    
map_singapore

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'BULHMQR1TRTERQPCD0MNSVPI2FPNV3DZ1YCMRYUT4VVZ0VPP' # your Foursquare ID
CLIENT_SECRET = '1X2NBGI2M4ZFTYADR4XXTG0HG2KX0UXU43XTTLIJQ0PNVKMO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## 3. Explore regions in Singapore

#### Let's create a function to get the venues to all the regions in Singapore

Exploring the regions
1. Create the get request url (Foursquare ID and Secret are necessary) 
    1.a. Number of Venues we will look for is 100 2.a. Radius of Search Would be 1 k.m. 
2. Create a json from the request object (Need requests Module) 
3. Create the lists Containing all the information 
4. From the lists create the dataframe.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    LIMIT = 100 # limit of number of venues returned by Foursquare API

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood_Latitude', 
                  'Neighborhood_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

### Use the Function Above to Create the Dataframe of Venues Around areas of Singapore


In [ ]:
singapore_Venues = getNearbyVenues(names=Singapore_data_Final['Name'],
                                   latitudes=Singapore_data_Final['Latitude'],
                                   longitudes=Singapore_data_Final['Longitude']
                                  )



#### Let's check the size of the resulting dataframe

In [ ]:
print(singapore_Venues.shape)
singapore_Venues.head()

In [ ]:
# Create a Data-Frame out of it to Concentrate Only on Restaurants 

Singapore_Venues_only_restaurant = singapore_Venues[singapore_Venues['Venue_Category']\
                                                          .str.contains('Restaurant')].reset_index(drop=True)
Singapore_Venues_only_restaurant.index = np.arange(1, len(Singapore_Venues_only_restaurant)+1)
print ("Shape of the Data-Frame with Venue Category only Restaurant: ", Singapore_Venues_only_restaurant.shape)
Singapore_Venues_only_restaurant.head(3)

In [ ]:
## Show in Map the Top Rated Restaruants 

map_restaurants = folium.Map(location=[latitude, longitude], zoom_start=11, tiles="openstreetmap", 
                            attr="<a href=https://github.com/python-visualization/folium/>Folium</a>")

# set color scheme for the Venues based on the Major Districts
Area_Name = Singapore_data_Final["Name"]

x = np.arange(len(Area_Name))

#rainbow = ['#00ff00', '#ff00ff','#0000ff','#ffa500' ,'#ff0000']



# add markers to the map
# markers_colors = []
for lat, lon, poi, distr in zip(Singapore_Venues_only_restaurant['Venue_Latitude'], 
                                  Singapore_Venues_only_restaurant['Venue_Longitude'], 
                                  Singapore_Venues_only_restaurant['Venue_Category'], 
                                  Singapore_Venues_only_restaurant['Neighborhood']):
    label = folium.Popup(str(poi) + ' ' + str(distr), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
 #       color=rainbow[Area_Name.index(distr)-1],
        fill=True,
  #      fill_color=rainbow[Area_Name.index(distr)-1],
        fill_opacity=0.3).add_to(map_restaurants)
       
map_restaurants

Let's check how many venues were returned for each neighborhood

In [ ]:
### Number of Unique Categories in the Dataframe 
print('There are {} unique categories.'.format(len(Singapore_Venues_only_restaurant['Venue_Category'].unique())))


We have seen that there are 59 unique categories in the Venues Data-Frame. 
Let's see the Frequency of Each Category


In [ ]:
print (Singapore_Venues_only_restaurant['Venue_Category'].value_counts())

### Create a Data-frame with the 10 Most Frequently Occuring Venue_Category


In [ ]:
# create a dataframe of top 10 categories
singapore_Venues_Top10 = Singapore_Venues_only_restaurant['Venue_Category'].value_counts()[0:10].to_frame(name='frequency')
singapore_Venues_Top10=singapore_Venues_Top10.reset_index()
#Singapore_Venues_Top10

singapore_Venues_Top10.rename(index=str, columns={"index": "Venue_Category", "frequency": "Frequency"}, inplace=True)
singapore_Venues_Top10

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(18,7))
s=sns.barplot(x="Venue_Category", y="Frequency", data=singapore_Venues_Top10)
s.set_xticklabels(s.get_xticklabels(), rotation=30)
plt.title('10 Most Frequently Occuring Venues in Singapore', fontsize=15)
plt.xlabel("Venue Category", fontsize=15)
plt.ylabel ("Frequency", fontsize=15)
plt.savefig("Most_Freq_Venues.png", dpi=300)
plt.show()


## Chinese Restaurant is mostly frequent venues in Singapore 

In [ ]:
Singapore_Venues_only_restaurant.groupby('Neighborhood').count()

## 4. Analyze Each Neighborhood

In [ ]:
# one hot encoding
singapore_onehot = pd.get_dummies(Singapore_Venues_only_restaurant[['Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Neighborhood'] = Singapore_Venues_only_restaurant['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()

And let's examine the new dataframe size.

In [ ]:
singapore_onehot.shape

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
singapore_grouped = singapore_onehot.groupby('Neighborhood').mean().reset_index()
singapore_grouped

#### Let's confirm the new size

In [ ]:
singapore_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in singapore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = singapore_grouped[singapore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
neighborhoods_venues_sorted

## 5. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

singapore_grouped_clustering = singapore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

singapore_merged = Singapore_data_Final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
singapore_merged = singapore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')

singapore_merged.head() # check the last columns!

In [ ]:
singapore_merged['ClusterLabels'].fillna(0, inplace=True)
singapore_merged['ClusterLabels'] = singapore_merged['ClusterLabels'].apply(np.int64)
singapore_merged['ClusterLabels']

Finally, let's visualize the resulting clusters

In [ ]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['Name'], singapore_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters


#### Cluster 1

In [ ]:
singapore_cluster1=singapore_merged.loc[singapore_merged['ClusterLabels'] == 1, singapore_merged.columns[[0] + list(range(1, singapore_merged.shape[1]))]]
print ("No of Neighbourhood in Cluster Label 1: %d" %(singapore_cluster1.shape[0]))
singapore_cluster1

#### Cluster 2

In [ ]:
singapore_cluster2=singapore_merged.loc[singapore_merged['ClusterLabels'] == 2, singapore_merged.columns[[0] + list(range(1, singapore_merged.shape[1]))]]
print ("No of Neighbourhood in Cluster Label 2: %d" %(singapore_cluster2.shape[0]))
singapore_cluster2

#### Cluster 3

In [ ]:
singapore_cluster3=singapore_merged.loc[singapore_merged['ClusterLabels'] == 3, singapore_merged.columns[[0] + list(range(1, singapore_merged.shape[1]))]]
print ("No of Neighbourhood in Cluster Label 3: %d" %(singapore_cluster3.shape[0]))
singapore_cluster3

#### Cluster 4

In [ ]:
singapore_cluster4=singapore_merged.loc[singapore_merged['ClusterLabels'] == 4, singapore_merged.columns[[0] + list(range(1, singapore_merged.shape[1]))]]
print ("No of Neighbourhood in Cluster Label 4: %d" %(singapore_cluster4.shape[0]))
singapore_cluster4

## 7. Results and Discussion:
We reached at the end of the analysis, where we got a sneak peak of the different areas of Singapore  and, as the business problem started with benefits and drawbacks of opening a lunch restaurant in one of the busiest areas, the data exploration was mostly concentrated on the restaurants. I have used data from web resources like Wikipedia, python libraries like Geopy, and Foursquare API, to set up a very realistic data-analysis scenario. We have found out that —

1. Chinese restaurants top the charts of most common venues in Singapore.
2. Geylang (Central region) and Novena (Central region) are dominated by restaurants as the the most common venue.

According to this analysis, central region is highly populated with both commercial and residential area. Furthermore, this results also could potentially vary if we use some other clustering techniques like DBSCAN

## 8. Conclusion

Finally to conclude this project, We have got a small glimpse of how real life data-science projects look like. I’ve made use of some frequently used python libraries to scrap web-data, use Foursquare API to explore the areas of Singapore and saw the results of segmentation of districts using Folium leaflet map. Potential for this kind of analysis in a real life business problem is discussed in great detail. Also, some of the drawbacks and chance for improvements to represent even more realistic pictures are mentioned. Finally, since my analysis were mostly concentrated on the possibilities of opening a restaurants targeting the huge pool of office workers. Hopefully, this kind of analysis will provide you initial guidance to take more real-life challenges using data-science.